In [10]:
import yaml
import pandas as pd
from pathlib import Path
from requests import get
from datetime import datetime
from fhir.resources.observation import Observation
from fhir.resources.reference import Reference
from fhir.resources.extension import Extension
from survey_obs import survey_obs
from fhir.resources.fhirtypesvalidators import ValidationError



In [11]:

'''Dict that maps the column names to the FHIR Observation codes for each LHC-Forms question'''
out_dir = r'/Users/ehaas/Documents/FHIR/SAIG/input/examples-yaml'

In [12]:
def getDisplayText(my_code):
    '''Get the display text for the LOINC code'''
    headers = {'Authorization':"Basic ZWhhYXM6bG9pbmM="}
    url = f'https://fhir.loinc.org/CodeSystem/$lookup?system=http://loinc.org&code={my_code}'
    response = get(url, headers=headers)
    if response.status_code == 200:
        display = next(d.get('valueString') for d in response.json()['parameter'] if d.get('name') == 'display')
        return display
    else:
        return 'Unknown'
# getDisplayText(55757-9)

In [13]:
def write_yaml(fhir_obj):
    out_path = Path(out_dir) / f'Observation-{fhir_obj.id}.yml'
    # note need to the date time format is not json serializabel out of the box see QR-OBs-DE/YAML-JSON-date-serialization.ipynb
    out_path.write_text(fhir_obj.yaml())

In [14]:
'''Using the fhir.resources to create the Observation resource
create panel and items Observations for each row of data in the CSV file
as follows:

1. panel with members that are the items and other panels
'''

def create_observation(row, df, code, category, example_id, items = None):

    my_display = getDisplayText(code.split('[')[0])
    obs = Observation.parse_file("survey_obs.yml")
    obs.id = f"{example_id}-{code.replace('[','answer-').replace(']','')}"
    obs.meta.extension[0].valueString = obs.id.title()
    obs.meta.extension[1].valueMarkdown = f'This is a an example of {code} (display = {my_display}) \
for the *US Core Observation Screening Assessment Profile*'
    obs.status = "final"
    obs.category = [{"coding": [{"system": "http://hl7.org/fhir/us/core/CodeSystem/us-core-category", 
                                 "code": category,}]}]
    if "CUST" in code:
        system = "http://example.org"
    else:
        system = "http://loinc.org"
    obs.code = {"coding": [{"system": system, "code": code, "display": my_display}]}
    obs.subject = {"reference": "Patient/" + df['Respondent_ID'][row]}
    obs.performer = [{"reference": "Performer/" + df['Author'][row]}]
    obs.effectiveDateTime =     mydate = df['Date'][row].isoformat() if "T00:00:00" not in df['Date'][row].isoformat() else df['Date'][row].isoformat()[:-9]
    # obs.derivedFrom[0] = {"reference": "DocumentReference/" + df['PDF'][row]}
    obs.derivedFrom = []
    #Check if panel
    obs.extension = []
    ext = Extension()
    ext.url = "http://www.fhir.org/guides/saig/StructureDefinition/observation-questionnaire"
    ext.valueCanonical = df['Canonical'][row]
    obs.extension.append(ext)
    if items:  # if this is a panel, then no valueCodeableConcep
        obs.hasMember = []
        for i in items:
            ref = Reference()
            ref.reference = f"Observation/{example_id}-{i.replace('[','answer-').replace(']','')}"
            ref.display = getDisplayText(i) if getDisplayText(i) != 'Unknown' else None
            obs.hasMember.append(ref)
            # create the individual Observations for the 3 questions
            # create_observation(df, panel_code=panel_code, item_code=i)
    else: # this is an individual question
        obs.hasMember = None
        value = df[code][row]
        if value =='nan': # then skip this observation
            return
        print(code, row, value)
        if '-' in value: 
            my_display = getDisplayText(value)
            if "CUST" in value:
                system = "http://example.org"
            else:
                system = "http://loinc.org"
            if my_display == 'Unknown':
                obs.valueCodeableConcept = {"coding": [{"system": system, "code": value}]} 
            else:
                obs.valueCodeableConcept = {"coding": [{"system": system, "code": value, "display": my_display}]}
        else:
            try:
                obs.valueQuantity = {"value": value}
            except ValidationError as e:
                print(e)
                obs.valueString = value
    write_yaml(obs)

In [15]:
def check_panel_code(lhc_code):
    filter = (df == lhc_code).any()
    sub_df = df.loc[: , filter]
    sub_df

In [16]:
def get_df(my_csv):
    df = pd.read_csv(my_csv).astype(str)
    df['Date']= pd.to_datetime(df['Date'])
    return df

In [17]:
### raw to disable
def main():
    # my_csv = "/Users/ehaas/Documents/FHIR/SAIG/input/images/Example1PHQ-2-55757-9.csv"
    my_csv = "/Users/ehaas/Documents/FHIR/SAIG/input/images/Example2CustomPHQ-2.csv"
    df = get_df(my_csv)
    print(df)
    # create the PHQ-2 panel
    # level1 = {'code':'55757-9', 'category': 'disability-status', 'itemz': ('44250-9','44255-8','55758-7')}
    level1 = {'code':'CUST-PNL', 'category': 'disability-status', 'itemz': ('44250-9','44255-8','55758-7', 'CUST-ITM')}
    # create the PHQ-2 panel
    # create level1 observations
    row = 1
    # example_id = 'CSV-PHQ-2Example1'
    example_id = 'CSV-CustomFormExample2'
    #get level 1 panel
    code = level1['code']
    category = level1['category']
    items = [item for item in level1['itemz'] if df[item][row] != 'nan']
    create_observation(row, df, code , category, example_id, items)
    #get level 1 answer codes
    for i in level1['itemz']:
        create_observation(row, df, i,  category ,example_id,)
        
if __name__ == "__main__":
    main()

  Respondent_ID       Date    Author    Location  \
0           123 2021-01-01    rsmith    New York   
1       example 2021-02-02  example2  California   
2           456 2021-03-03    jsmith       Texas   

                                PDF  \
0  example1customphq-2questionnaire   
1  example1customphq-2questionnaire   
2  example1customphq-2questionnaire   

                                           Canonical CUST-PNL 44250-9 44255-8  \
0  http://www.fhir.org/guides/saig/Questionnaire/...      nan       0       0   
1  http://www.fhir.org/guides/saig/Questionnaire/...      nan       1       1   
2  http://www.fhir.org/guides/saig/Questionnaire/...      nan       2       2   

  55758-7 CUST-ITM  
0       0      red  
1       2     blue  
2       4      red  
44250-9 1 1
44255-8 1 1
55758-7 1 2
CUST-ITM 1 blue
1 validation error for Observation
valueQuantity -> value
  value is not a valid decimal (type=type_error.decimal)
